# **FGV EESP**

- ## Dados Econômicos - Brasil
---



### **Bibliotecas**

In [ ]:
# Obter os dados do SGS do Banco Central do Brasil
install.packages("rbcb")

# Salvar dados em formato .xlsx
install.packages("writexl")

In [ ]:
library(dplyr)
library(httr)
library(jsonlite)
library(lubridate)
library(readxl)
library(rbcb)
library(tidyverse)
library(writexl)
library(zoo)

### **Dados**

- #### **Expectativa de Inflação do Mercado**

  * Referência: [Olinda BCB](https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!/recursos/ExpectativasMercadoInflacao12Meses#eyJmb3JtdWxhcmlvIjp7IiRmb3JtYXQiOiJqc29uIiwiJHRvcCI6MTAwLCIkZmlsdGVyIjoiBDkEIGVxIDAgYW5kIAQyBCBlcSAnUycgYW5kIAQwBCBlcSAnSVBDQScgYW5kIAQxBCBnZSAnMjAwNi0wMS0wMScgYW5kIAQxBCBsZSAnMjAyMy0wNi0zMCciLCIkb3JkZXJieSI6IgQxBCBkZXNjIn0sInByb3ByaWVkYWRlcyI6WzAsMSwzLDQsNV0sInBlc3F1aXNhZG8iOnRydWUsImFjdGl2ZVRhYiI6InRhYmxlIiwiZ3JpZFN0YXRlIjp7AzADOlt7A0IDIgQwBCIsA0EDfSx7A0IDIgQxBCIsA0EDfSx7A0IDIgQzBCIsA0EDfSx7A0IDIgQzBG5hIiwDQQN9LHsDQgMiBDUEIiwDQQN9XSwDMQM6e30sAzIDOltdLAMzAzp7fSwDNAM6e30sAzUDOnt9fSwicGl2b3RPcHRpb25zIjp7A2EDOnt9LANiAzpbXSwDYwM6NTAwLANkAzpbIgQxBCJdLANlAzpbXSwDZgM6WyJN6WRpYSJdLANnAzoidmFsdWVfel90b19hIiwDaAM6InZhbHVlX3pfdG9fYSIsA2kDOnt9LANqAzp7fSwDawM6ODUsA2wDOmZhbHNlLANtAzp7fSwDbgM6e30sA28DOiJN6WRpYSIsA3ADOiJMaW5lIENoYXJ0In19)

In [ ]:
# Definir a URL da API do BCB para obter dados de Expectativa do Mercado para IPCA nos próximos 12 meses
url <- "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoInflacao12Meses?$filter=baseCalculo%20eq%200%20and%20Suavizada%20eq%20'S'%20and%20Indicador%20eq%20'IPCA'%20and%20Data%20ge%20'2005-12-01'%20and%20Data%20le%20'2023-06-30'&$orderby=Data%20desc&$format=json&$select=Indicador,Data,Media,Mediana,DesvioPadrao"

# Realizar a consulta e ler o arquivo JSON
response <- GET(url)
data_json <- rawToChar(response$content)

# Ajuste das colunas de data
focus_inflation_expectation <- fromJSON(data_json)$value %>%
  mutate(day = ymd(Data),
         month_date = floor_date(day, unit = "month"),
         year = year(month_date),
         day_of_week = weekdays(day),
         month_name = month(month_date, label = TRUE, abbr = TRUE))

# Filtrar apenas as sextas-feiras (Referência do Boletim Focus)
fridays_only <- focus_inflation_expectation %>%
  filter(day_of_week == "Friday")

# Calcular média da coluna "Mediana" agrupada por "year" e "month_date"
result <- fridays_only %>%
  group_by(year, month_date, month_name) %>%
  summarize(market_inflation_expectation = round(mean(Mediana), 2))

# Conversão em formato de data frame e time series
df_olinda <- as.data.frame(ts(result$market_inflation_expectation, start = c(2005, 12), frequency = 12))
df_olinda <- rename(df_olinda, market_inflation_expectation = x)

# Criação de colunas para mês e ano
df_olinda$month = time(df_olinda$market_inflation_expectation)
df_olinda$month_date = result$month_date
df_olinda$month_name = result$month_name
df_olinda$year = result$year

`summarise()` has grouped output by 'year', 'month_date'. You can override
using the `.groups` argument.


- #### **Expectativa de Inflação dos Consumidores**

  * Referência: [FGV IBRE](https://extra-ibre.fgv.br/IBRE/sitefgvdados/default.aspx)

In [ ]:
# Indicador de Expectativa de Inflação dos Consumidores
# Código FGV Dados: 1416188

# Caminho do arquivo original
file_path <- "https://github.com/matheusrmorgado/inflation-expectation/raw/main/data/fgv_ibre.xlsx"

# Download do arquivo
download.file(file_path, "fgv_ibre.xlsx")

# Ler o arquivo Excel e converter em dataframe
df_fgv <- as.data.frame(read_excel("fgv_ibre.xlsx"))

# Filtrar os dados de 2005-12-01 até 2023-06-01
df_fgv <- df_fgv %>% filter(as.Date(month_date) >= as.Date("2005-12-01") & as.Date(month_date) <= as.Date("2023-06-01"))

# Conversão em formato timeseries
df_consumer <- as.data.frame(ts(df_fgv$consumer_inflation_expectation, start = c(2005, 12), frequency = 12))
df_consumer <- rename(df_consumer, consumer_inflation_expectation = x)

- #### **Dados Macroeconômicos**

  * Referência: [SGS BCB](https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries)

In [ ]:
# Índice nacional de preços ao consumidor amplo acumulado em 12 meses (%)
# SGS: 13522
df_ipca <- as.data.frame(get_series(13522, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts"))
df_ipca <- rename(df_ipca, ipca = x)

# Taxa de juros Selic acumulada no mês anualizada base 252 (% a.a.)
# SGS: 4189
df_selic <- as.data.frame(get_series(4189, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts"))
df_selic <- rename(df_selic, selic = x)

# Índice de Atividade Econômica do Banco Central com ajuste sazonal (índice)
# SGS: 24364
df_ibc_br <- as.data.frame(get_series(24364, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts"))
df_ibc_br <- rename(df_ibc_br, ibc_br = x)

# Dívida Líquida do Governo Geral (% PIB)
# SGS: 4536
df_gov_debt <- as.data.frame(get_series(4536, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts"))
df_gov_debt <- rename(df_gov_debt, gov_debt = x)

# Endividamento das famílias com o SFN em relação à renda acumulada dos últimos 12 meses (%)
# SGS: 29037
df_household_debt <- as.data.frame(get_series(29037, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts"))
df_household_debt <- rename(df_household_debt, household_debt = x)

# RNDBF restrita - valores correntes, acumulada 12 meses - R$ (bilhões)/
# SGS: 7836
df_household_income <- as.data.frame(get_series(7836, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts"))
df_household_income <- rename(df_household_income, household_income = x)

# PIB acumulado dos últimos 12 meses - Valores correntes - R$ (bilhões)
# SGS: 4382
df_pib <- as.data.frame(get_series(4382, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts"))
df_pib <- rename(df_pib, pib = x)

# Taxa de câmbio - Livre - Dólar americano (compra) - Fim de período - mensal (u.m.c./US$)
# SGS: 3695
df_usd_brl <- as.data.frame(get_series(3695, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts"))
df_usd_brl <- rename(df_usd_brl, usd_brl = x)

- #### **União de todas as bases de dados**

In [ ]:
# Realiza a unificação de todas as tabelas do BCB e FGV IBRE
df_public_data <- cbind(df_olinda, df_consumer, df_ipca, df_selic, df_ibc_br, df_gov_debt, df_household_debt, df_household_income, df_pib, df_usd_brl)

# Calcula o lag para IPCA e Selic
df_public_data$ipca_lag <- c(NA, df_public_data$ipca[-nrow(df_public_data)])
df_public_data$selic_lag <- c(NA, df_public_data$selic[-nrow(df_public_data)])

# Converter as colunas "ipca_lag" e "selic_lag" para séries temporais
df_public_data$ipca_lag <- ts(df_public_data$ipca_lag, start = c(2005, 12), frequency = 12)
df_public_data$selic_lag <- ts(df_public_data$selic_lag, start = c(2005, 12), frequency = 12)

# Reorganiza as colunas, colocando "month_date" e "month" nas primeiras posições
df_public_data <- df_public_data %>% dplyr::select(year, month_name, month_date, month, everything())

In [ ]:
# Identificar as variáveis que são Time-Series
ts_cols <- sapply(df_public_data, is.ts)

# Filtrar o dataframe
df_public_data <- df_public_data %>%
  filter(month_date >= as.Date("2006-01-01"))

# Transformar as colunas identificadas como Time-Series de volta em Time-Series
df_public_data[ts_cols] <- lapply(df_public_data[ts_cols], function(x) ts(x, start = c(2006, 1), frequency = 12))

- #### **Visualização da base de dados**

In [ ]:
str(df_public_data)

'data.frame':	210 obs. of  16 variables:
 $ year                          : num  2006 2006 2006 2006 2006 ...
 $ month_name                    : Ord.factor w/ 12 levels "Jan"<"Feb"<"Mar"<..: 1 2 3 4 5 6 7 8 9 10 ...
 $ month_date                    : Date, format: "2006-01-01" "2006-02-01" ...
 $ month                         : Time-Series  from 2006 to 2023: 2006 2006 2006 2006 2006 ...
 $ market_inflation_expectation  : Time-Series  from 2006 to 2023: 4.58 4.49 4.36 4.16 4.18 4.25 4.4 4.48 4.29 4.05 ...
 $ consumer_inflation_expectation: Time-Series  from 2006 to 2023: 6.9 7 6.9 6.5 6.7 6.3 6.9 6.4 6.3 5.8 ...
 $ ipca                          : Time-Series  from 2006 to 2023: 5.7 5.51 5.32 4.63 4.23 4.03 3.97 3.84 3.7 3.26 ...
 $ selic                         : Time-Series  from 2006 to 2023: 17.6 17.3 16.7 16.2 15.7 ...
 $ ibc_br                        : Time-Series  from 2006 to 2023: 114 114 113 114 116 ...
 $ gov_debt                      : Time-Series  from 2006 to 2023: 46.7 46

In [ ]:
head(df_public_data, 5)

,year,month_name,month_date,month,market_inflation_expectation,consumer_inflation_expectation,ipca,selic,ibc_br,gov_debt,household_debt,household_income,pib,usd_brl,ipca_lag,selic_lag
,<dbl>,<ord>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2006,Jan,2006-01-01,2006.000,4.58,6.9,5.70,17.65,113.56,46.68,19.89,167991,2190643,2.2152,5.69,18.24
2,2006,Feb,2006-02-01,2006.083,4.49,7.0,5.51,17.28,113.72,46.38,20.11,169200,2207671,2.1347,5.70,17.65
3,2006,Mar,2006-03-01,2006.167,4.36,6.9,5.32,16.74,113.48,46.38,20.45,166456,2225145,2.1716,5.51,17.28
4,2006,Apr,2006-04-01,2006.250,4.16,6.5,4.63,16.19,114.33,45.59,20.78,165894,2235813,2.0884,5.32,16.74
5,2006,May,2006-05-01,2006.333,4.18,6.7,4.23,15.70,115.65,46.21,21.07,165256,2256419,2.2997,4.63,16.19


- #### **Salvar em formato `.csv` e `.xlsx`**

In [ ]:
# Salve o DataFrame como um arquivo CSV localmente
write.csv(df_public_data, file = "public_data.csv", row.names = FALSE)

In [ ]:
# Salve o DataFrame como um arquivo XLSX localmente
write_xlsx(df_public_data, path = "/content/public_data.xlsx")